In [39]:
from functools import reduce
from flore.tree import ID3, FDT
from flore.tree.tests.fdt_legacy_tree import FDT_Legacy
from flore.tree.tests.id3_legacy_tree import ID3_Legacy
from pytest import fixture

from sklearn import datasets
from sklearn.model_selection import train_test_split

from flore.fuzzy import get_fuzzy_points, get_fuzzy_variables, get_dataset_membership
from flore.datasets import load_compas, load_beer
from flore.explanation import FID3_factual, m_factual, mr_factual, c_factual
import numpy as np
import pandas as pd
import random
import xml.etree.cElementTree as e

In [11]:
from py4jfml.FuzzyInferenceSystem import FuzzyInferenceSystem
from py4jfml.Py4Jfml import Py4jfml
import os

from py4jfml.knowledgebase.KnowledgeBaseType import KnowledgeBaseType
from py4jfml.knowledgebasevariable.FuzzyVariableType import FuzzyVariableType
from py4jfml.rule.AntecedentType import AntecedentType
from py4jfml.rule.ClauseType import ClauseType
from py4jfml.rule.ConsequentType import ConsequentType
from py4jfml.rule.FuzzyRuleType import FuzzyRuleType
from py4jfml.rulebase.MamdaniRuleBaseType import MamdaniRuleBaseType
from py4jfml.term.FuzzyTermType import FuzzyTermType

In [3]:
seed = 0
random.seed(seed)
np.random.seed(seed)

In [4]:
def _get_fuzzy_element(fuzzy_X, idx):
    element = {}
    for feat in fuzzy_X:
        element[feat] = {}
        for fuzzy_set in fuzzy_X[feat]:
            try:
                element[feat][str(fuzzy_set)] = pd.to_numeric(fuzzy_X[feat][fuzzy_set][idx])
            except ValueError:
                element[feat][str(fuzzy_set)] = fuzzy_X[feat][fuzzy_set][idx]

    return element

In [5]:
dataset = load_beer()

df = dataset['df']
class_name = dataset['class_name']
X = df.drop(class_name, axis=1)
y = df[class_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

df_categorical_columns = dataset['discrete']
class_name = dataset['class_name']
df_categorical_columns.remove(class_name)
df_numerical_columns = dataset['continuous']

df_train = df.loc[X_train.index]
df_test = df.loc[X_test.index]

fuzzy_points = get_fuzzy_points(df_train, 'entropy', df_numerical_columns, class_name=class_name)
discrete_fuzzy_values = {col: df_train[col].unique() for col in df_categorical_columns}
fuzzy_variables = get_fuzzy_variables(fuzzy_points, discrete_fuzzy_values)
df_train_membership = get_dataset_membership(df_train, fuzzy_variables)
df_test_membership = get_dataset_membership(df_test, fuzzy_variables)

fuzzy_element = _get_fuzzy_element(df_test_membership, 48)
all_classes = dataset['possible_outcomes']

In [6]:
new_fdt = FDT(df_train_membership.keys())
new_fdt.fit(df_train_membership, y_train.to_numpy())
rules = new_fdt.to_rule_based_system()

In [92]:
membership_function_names = ['MF0', 'MF1', 'MF2', 'MF3', 'MF4', 'MF5', 'MF6', 'MF7', 'MF8', 'MF9']
xml_name = 'POTATO'


In [104]:
fuzzy_variables[0]

FuzzyVariable(name='color', fuzzy_sets=[FuzzyContinuousSet(name='0.0', fuzzy_points=[0.0, 0.0, 4.0]), FuzzyContinuousSet(name='4.0', fuzzy_points=[0.0, 4.0, 5.0]), FuzzyContinuousSet(name='5.0', fuzzy_points=[4.0, 5.0, 7.0]), FuzzyContinuousSet(name='7.0', fuzzy_points=[5.0, 7.0, 12.0]), FuzzyContinuousSet(name='12.0', fuzzy_points=[7.0, 12.0, 16.0]), FuzzyContinuousSet(name='16.0', fuzzy_points=[12.0, 16.0, 23.0]), FuzzyContinuousSet(name='23.0', fuzzy_points=[16.0, 23.0, 29.0]), FuzzyContinuousSet(name='29.0', fuzzy_points=[23.0, 29.0, 45.0]), FuzzyContinuousSet(name='45.0', fuzzy_points=[29.0, 45.0, 45.0])])

In [108]:
var_mapping = {
    'color': 'Color',
    'strength': 'Strength',
    'bitterness': 'Bitterness',
    'beer_style': 'Beer-Style'
}

In [14]:
iris = FuzzyInferenceSystem("POTATO")

#KNOWLEDGE BASE
kb = KnowledgeBaseType()
iris.setKnowledgeBase(kb)

In [105]:
def get_fuzzy_variables_xml(fuzzy_variables, all_classes, class_name):
    kb = e.Element("knowledgeBase")
    mf_names = {}
    for fv in fuzzy_variables:
        attrs = {}
        attrs['name'] = var_mapping[fv.name]
        attrs['scale'] = ''
        attrs['domainleft'] = str(fv.fuzzy_sets[0].fuzzy_points[0])
        attrs['domainright'] = str(fv.fuzzy_sets[-1].fuzzy_points[-1])
        attrs['type'] = 'input'
        ffv = e.SubElement(kb, "fuzzyVariable", attrib=attrs)
        mf_names[fv.name] = {}
        for i, fs in enumerate(fv.fuzzy_sets):
            ft_attrs = {}
            ft_attrs['name'] = membership_function_names[i]
            mf_names[fv.name][fs.name] = membership_function_names[i]
            ft_attrs['complement'] = 'false'
            ft = e.SubElement(ffv, "fuzzyTerm", attrib=ft_attrs)
            ts_attrs = {}
            ts_attrs['param1'] = str(fs.fuzzy_points[0])
            ts_attrs['param2'] = str(fs.fuzzy_points[1])
            ts_attrs['param3'] = str(fs.fuzzy_points[1])
            ts_attrs['param4'] = str(fs.fuzzy_points[2])
            e.SubElement(ft, "trapezoidShape", attrib=ts_attrs)

    attrs = {}
    attrs['name'] = var_mapping[class_name]
    attrs['scale'] = ''
    attrs['domainleft'] = '1.0'
    attrs['domainright'] = str(float(len(all_classes)))
    attrs['type'] = 'output'
    attrs['accumulation'] = 'MAX'
    attrs['defuzzifier'] = 'MOM'
    attrs['defaultValue'] = '1.0'
    cv = e.SubElement(kb, "fuzzyVariable", attrib=attrs)

    ft_attrs = {}
    ft_attrs['name'] = all_classes[0]
    ft_attrs['complement'] = 'false'
    cft = e.SubElement(cv, "fuzzyTerm", attrib=ft_attrs)
    ts_attrs = {}
    ts_attrs['param1'] = str("1.0")
    ts_attrs['param2'] = str("1.0")
    ts_attrs['param3'] = str("2.0")
    e.SubElement(cft, "triangleShape", attrib=ts_attrs)

    for i, cl in enumerate(all_classes[1:-1]):
        ft_attrs = {}
        ft_attrs['name'] = cl
        ft_attrs['complement'] = 'false'
        cft = e.SubElement(cv, "fuzzyTerm", attrib=ft_attrs)
        ts_attrs = {}
        ts_attrs['param1'] = str(float(i+1))
        ts_attrs['param2'] = str(float(i+2))
        ts_attrs['param3'] = str(float(i+3))
        e.SubElement(cft, "triangleShape", attrib=ts_attrs)
    
    ft_attrs = {}
    ft_attrs['name'] = all_classes[-1]
    ft_attrs['complement'] = 'false'
    cft = e.SubElement(cv, "fuzzyTerm", attrib=ft_attrs)
    ts_attrs = {}
    ts_attrs['param1'] = str(float(len(all_classes)-1))
    ts_attrs['param2'] = str(float(len(all_classes)))
    ts_attrs['param3'] = str(float(len(all_classes)))
    e.SubElement(cft, "triangleShape", attrib=ts_attrs)
    
    return kb, mf_names

In [82]:
rules[0].antecedent

(('color', '0.0'), ('bitterness', '8.0'))

In [106]:
def get_rules_xml(rule_list, class_name, mf_names):
    rb_attrs = {}
    rb_attrs['name'] = 'rulebase'
    rb_attrs['activationMethod'] = 'MIN'
    rb_attrs['andMethod'] = 'MIN'
    rb_attrs['orMethod'] = 'MAX'
    rb = e.Element("mamdaniRuleBase", attrib=rb_attrs)

    for i, rule in enumerate(rule_list):
        r_attrs = {}
        r_attrs['name'] = f'rule{i+1}'
        r_attrs['andMethod'] = 'MIN'
        r_attrs['connector'] = 'and'
        r_attrs['weight'] = str(rule.weight)
        r = e.SubElement(rb, "rule", r_attrs)
        ante = e.SubElement(r, 'antecedent')
        for var, term in rule.antecedent:
            clause = e.SubElement(ante, 'clause')
            xml_var = e.SubElement(clause, 'variable')
            xml_var.text = var_mapping[var]
            xml_term = e.SubElement(clause, 'term')
            xml_term.text = mf_names[var][term]
        
        conse = e.SubElement(r, 'consequent')
        then = e.SubElement(conse, 'then')
        clause = e.SubElement(then, 'clause')
        xml_var = e.SubElement(clause, 'variable')
        xml_var.text = var_mapping[class_name]
        xml_term = e.SubElement(clause, 'term')
        xml_term.text = rule.consequent
    
    return rb
    

In [109]:
fv_tree, mf_names = get_fuzzy_variables_xml(fuzzy_variables, all_classes, class_name)
rules_tree = get_rules_xml(rules, class_name, mf_names)


In [110]:
root_attrs = {}
root_attrs['xmlns'] = 'http://www.ieee1855.org'
root_attrs['name'] = xml_name
root = e.Element("fuzzySystem", attrib=root_attrs)
root.append(fv_tree)
root.append(rules_tree)

In [111]:
a = e.ElementTree(root)
a.write('test.xml', xml_declaration=True, encoding='UTF8')